# PARAMETERS 

In [8]:
problem_name = 'pills' #to save
weights_path = None 
target_size = (224, 224) 
tam_image = 224
batch_size = 1
epochs = 100 #após x épocas sem melhorar pará (a usar callback)

#just for tests (select max to train)
max_drImages = 100  #max 2000
max_dcImages = 0     #max 5000

In [9]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet201
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
from keras.constraints import maxnorm 
from keras.optimizers import SGD 
from keras.utils import np_utils 
from keras import backend as K 
from keras.utils.data_utils import get_file
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import keras
K.set_image_dim_ordering('tf') #ordem 'th' ou 'tf' 
import numpy as np
from numpy import genfromtxt
import math 

from timeit import default_timer as timer
from time import time as tick
import matplotlib.pyplot as plt 
import pickle 
from os import listdir
from PIL import Image, ImageOps
from os.path import isfile, join
import os
from scipy.misc	import toimage 
from scipy import misc, ndimage
import scipy.fftpack as pack
import scipy.misc
from scipy.ndimage import rotate
from sklearn.datasets import fetch_mldata
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from skimage.io import imread
from skimage.transform import resize

from sklearn.metrics import roc_auc_score
import pandas as pd
import pathlib

# fixar random seed para se puder reproduzir os resultados 
seed = 9 
np.random.seed(seed) 

/home/programs/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Útils

In [10]:
def print_model(model,fich):
    from keras.utils import plot_model
    plot_model(model, to_file=fich, show_shapes=True, show_layer_names=True)
    
def print_history_accuracy(history):
    print(history.history.keys())
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
def print_history_loss(history):
    print(history.history.keys())
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

def load_batch(fpath, label_key='labels'): 
 
    f = open(fpath, 'rb') 
    d = pickle.load(f, encoding='bytes') 
    d_decoded = {}        # decode utf8 
    for k, v in d.items(): 
        d_decoded[k.decode('utf8')] = v 
    d = d_decoded 
    f.close() 
    data = d['data'] 
    labels = d[label_key] 
    data = data.reshape(data.shape[0], 3, 32, 32) 
    return data, labels

def imageProcess(temp, tam_image):
    #-------------------rodar se necessário e cortar em quadrado
    if temp.shape[0] > temp.shape[1]:
        temp = rotate(temp,90)
    
    #cortar em quadrado no centro da imagem e fazer resize para o tam_image
    difShapes = temp.shape[1]-temp.shape[0]
    return (255 * resize(temp[0:temp.shape[0],int(difShapes/2):int(difShapes/2)+temp.shape[0]],
                            (tam_image, tam_image))).astype(np.uint8)

# preprocessing

In [11]:
print('a ler imagens de treino...')

num_images = max_drImages + max_dcImages      #max = 5000 + 1020
num_imagens_test = num_images/2
x_train = np.zeros((int(num_images), tam_image, tam_image, 3), dtype='uint8')
#y_train = np.zeros((int(num_images),1), dtype='uint8')
y_train_temp = ["" for x in range(num_images)]
groundTruth = genfromtxt('groundTruthTable.csv', delimiter=',', dtype=None)

#dr - images with more quality, dc - smartphone images

#read image names to array
drImages = sorted([f for f in listdir('dr/') if isfile(join('dr/', f))])

if num_images > max_drImages:  #just to set forMax (for tests I use just some images)
    forMax = max_drImages
else:
    forMax = num_images

#read dr images
for i in range(0, max_drImages): 
    tempdr = imread('dr/' + drImages[i])
    
    x_train[i,:,:,:] = imageProcess(tempdr, tam_image)
    y_train_temp[i],trash = drImages[i].rsplit('_', 1)[1].split(".") #split label  

#read dc images                                   
for i in range(max_drImages, num_images):
    a,b,c = str(groundTruth[i-max_drImages+1, 1]).split("'") #just to remove chars ''
    tempdc = imread('dc/' + b)
    
    x_train[i,:,:,:] = imageProcess(tempdc, tam_image)
    #label
    a,b,c = str(groundTruth[i-max_drImages+1, 0]).split("'") #just to remove chars ''
    y_train_temp[i],trash = b.rsplit('_', 1)[1].split(".") #split label 

#y_train[:,0] = LabelEncoder().fit_transform(y_train) #pass to 0,1,2,3... int labels
#y_train = y_train.astype('uint8')

y_train = np.zeros((int(num_images),1), dtype='uint16')
y_train[:,0] = LabelEncoder().fit_transform(y_train_temp) #pass to 0,1,2,3... int labels

print('Completo')
print('a ler imagens teste...')

x_test = np.zeros((int(num_imagens_test), tam_image, tam_image, 3), dtype='uint8')
y_test = np.zeros((int(num_imagens_test),1), dtype='uint16')

#read dr images
f=0
for i in range(0, int(num_imagens_test*2),2): 
    x_test[f,:,:,:] = x_train[i,:,:,:]
    y_test[f,:] = y_train[i,:]
    f=f+1
#y_test[:,0] = LabelEncoder().fit_transform(y_test) #pass to 0,1,2,3... int labels
#y_test = y_test.astype('uint8')
print('Completo')

a ler imagens de treino...


/home/programs/lib/python3.6/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  
/home/programs/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Completo
a ler imagens teste...
Completo


In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
#plots(x_train, figsize=(20,7), rows=2)
#plots(x_test, figsize=(20,7), rows=2)

(100, 224, 224, 3)
(50, 224, 224, 3)
(100, 1)
(50, 1)


# Models

In [13]:
#choosing model

def choosing_model(model_architecture, num_classes,epochs, weights_path=None):
    
    model = None
    if model_architecture == 'VGG_16':
        model = VGG_16(num_classes,epochs, weights_path=None)
        
    if model_architecture == 'ResNet50':    
        model = ResNet50(weights=None, classes=num_classes)
        
    if model_architecture == 'InceptionResNetV2':    
        model = InceptionResNetV2(weights=None, classes=num_classes)
        
    if model_architecture == 'Xception':    
        model = Xception(weights=None, classes=num_classes)
        
    if model_architecture == 'InceptionV3':    
        model = InceptionV3(weights=None, classes=num_classes)
        
    if model_architecture == 'DenseNet201':    
        model = DenseNet201(weights=None, classes=num_classes)
    #if model_architecture == 'VGG_19':
        #model = VGG_19(num_classes,epochs, weights_path=None)
        
    if model == None:
        print('non valid model')
    else:
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
            

In [14]:
def VGG_16(num_classes,epochs, weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))   

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)
        
    # Compile model 
    #lrate = 0.01 
    #decay = lrate/epochs 
    #sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False) 
    #model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) 
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Callbacks

In [15]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history_loss = LossHistory() #print(history.losses) to use      

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, mode='min')    

checkpoint = ModelCheckpoint(filepath = 'checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=False, mode='min', period=1)

#reduce training rate when no improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

csv_logger = CSVLogger('training.log')

# Training Functions

In [16]:
def training(model, model_architecture, train_generator, validation_generator, num_classes, epochs, batch_size, weights_path=None):
    
    print(model.summary())
    #print_model(model,"model_plus.png")
    
    history = model.fit_generator(
        train_generator, 
        steps_per_epoch = math.ceil(train_generator.n/batch_size),
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = math.ceil(validation_generator.n/batch_size),
        callbacks=[history_loss, early_stop, checkpoint, reduce_lr, csv_logger],
        verbose=1) 
    
    model.save(model, problem_name + '_' + model_architecture + '.h5') 
    
    print_history_accuracy(history) 
    print_history_loss(history) 
    # Final evaluation with test cases
    scores = model.evaluate_generator(validation_generator) 
    print('Scores: ', scores) 
    print("Accuracy: %.2f%%" % (scores[1]*100)) 
    print("Erro modelo: %.2f%%" % (100-scores[1]*100))
       
    return history 


def training_data(model_architecture, x_train, y_train, x_test, y_test, epochs, batch_size, weights_path=None):    
    
    num_classes = int(np.amax(y_train[:,0])+1)
    
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    
    #atention to type float32 uint8..
    
    train_datagen = ImageDataGenerator(      #https://keras.io/preprocessing/image/
        featurewise_center=True,
        #featurewise_std_normalization=True, .fit(numpy_data)
        rotation_range=20,
        zoom_range=[5,5],
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        rescale=1./255) 
    
    test_datagen = ImageDataGenerator(
        featurewise_center=True,
        #featurewise_std_normalization=True, .fit(numpy_data)
        rescale=1./255)

    
    train_generator = train_datagen.flow(
        x_train[0:max_drImages,:,:,:], 
        y_train[0:max_drImages,:], 
        batch_size=batch_size,
        shuffle=True)
        
    validation_generator = test_datagen.flow(
        x_test, 
        y_test, 
        batch_size=batch_size,
        shuffle=True)
    
    #CREATE MODEL
    model = choosing_model(model_architecture, num_classes, epochs, weights_path=None)
    
    problem_name = 'pillsdr'
    print('Train 1/2 - dr images')
    history = training(model, model_architecture, train_generator, validation_generator, num_classes, epochs, batch_size, weights_path)
    
    
    #re-train with dc data augmentation
    
    train_datagen = ImageDataGenerator(      #https://keras.io/preprocessing/image/
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=2,
        height_shift_range=2,
        horizontal_flip=True,
        rescale=1./255) 
    
    train_generator = train_datagen.flow(
        x_train[max_drImages:max_images,:,:,:], 
        y_train[max_drImages:max_images,:], 
        batch_size=batch_size,
        shuffle=True)
    
    problem_name = 'pillsdc_finaltrain'
    print('Train 2/2 - dc images')
    #retrain model with dc images (different data augmentation)
    history = training(model, model_architecture, train_generator, validation_generator, num_classes, epochs, batch_size, weights_path)
    
    return history, model

# Training

In [18]:
if __name__ == '__main__':
    
    archs = 'VGG_16', 'ResNet50', 'InceptionResNetV2', 'Xception', 'InceptionV3', 'DenseNet201'
    for i in range(0,6):
        model_architecture = archs[i]    
        checkpoint = ModelCheckpoint(filepath = model_architecture + '_ckpt-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=False, mode='min', period=1)
        model, history = training_data(model_architecture, x_train, y_train, x_test, y_test, epochs, batch_size, weights_path=None)


Train 1/2 - dr images
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_50 (Activation)      (None, None, None, 3 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

Epoch 1/100


/home/marcelo/.local/lib/python3.6/site-packages/keras/preprocessing/image.py:594: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


ResourceExhaustedError: OOM when allocating tensor with shape[1088,384,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block17_6_conv/convolution = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block17_6_mixed/concat, block17_6_conv/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics_1/acc/Mean/_18017 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_66352_metrics_1/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happen

# ----------------> NÃO EXECUTAR

# Testing

In [237]:
model_name = 'VGG_16'
testPath = 'test/'

model = load_model('Models/' + model_name + 'model.h5')

test_datagen = ImageDataGenerator(
    rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        testPath,
        color_mode = 'rgb',
        class_mode = 'categorical',
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)  # keep data in same order as labels


# Final evaluation with test cases
scores = model.evaluate_generator(validation_generator) 
print('Scores: ', scores) 
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("Erro modelo: %.2f%%" % (100-scores[1]*100))

scores = model.predict_generator(validation_generator)
j, predicted_classes = np.unravel_index(scores.argmax(axis=1), scores.shape)

for i in range(0, scores.shape[0]):
    if predicted_classes[i] != validation_generator.classes[i]:#print failed images
        plt.imshow(imread(pathtest + validation_generator.filenames[i]))
        plt.show()
        print('true: ', validation_generator.classes[i])
        print('prediction: ',predicted_classes[i])
        print('scores: ', scores[i])
        print('file: ', validation_generator.filenames[i])
    

OSError: Unable to open file (unable to open file: name = 'Models/VGG_16model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Confusion Matrix + AUC

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(validation_generator.classes, predicted_classes))

# AUC for prediction on validation sample
X_val_sample, val_labels = next(validation_generator)
val_pred = model.predict_proba(X_val_sample)
val_pred = np.reshape(val_pred, val_labels.shape)
val_score_auc = roc_auc_score(val_labels, val_pred)
print ("AUC validation score: ",val_score_auc)

# NÃO EXECUTAR

# Tests data_augmentation

In [ ]:
dir_to_save = 'testes'
n_imgs = 10
#select image 
origin = '/media/marcelo/OS/Users/Marcelo Queirós/Documents/MIEI/Semestre 2/Tecnologias e Aplicacoes/deep learning/Projects/pills/dc/' 
image_path = origin + '1.jpg'

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=2,
    height_shift_range=2,
    horizontal_flip=True)

image = np.expand_dims(ndimage.imread(image_path),0) 


# fits the model on batches with real-time data augmentation:
#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    #steps_per_epoch=len(x_train) / 32, epochs=epochs)
i = 0
for batch in datagen.flow(image, batch_size=1, 
                          save_to_dir = dir_to_save, save_prefix='dc', save_format='jpg'):
    i += 1
    if i > n_imgs:
        break  # otherwise the generator would loop indefinitely

In [ ]:
#plots(aug_images, figsize=(20,7), rows=2)